In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import datetime as dt

%config InteractiveShell.ast_node_interactivity = 'all'
#import dependencies

In [2]:
df = pd.read_parquet("../Proyecto-Taxis-NYC/data/yellow_t_2018-01.parquet", engine="pyarrow")

In [3]:
#esto se uso inicilamente para poder darnos una idea de los datos a tratar

"""
def get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):
    labels,values,percent = list(),list(),list()
    if df.isna().sum().sum()>0:
        for column in df.columns:
            if df[column].isna().sum():
                labels.append(column)
                values.append(df[column].isna().sum())
                percent.append((df[column].isna().sum() / df.shape[0]) * 100)
        #Make a dataframe 
        missings=pd.DataFrame({'Colúmnas':labels,'Valores faltantes':values,'PorcentajePerdido':percent}).sort_values(by='PorcentajePerdido',ascending=False)
        

        if plot:
            plt.figure(figsize=figsize)
            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title('Porcentaje de valores perdidos',size=22)
            for i in plot.patches:
                plot.annotate(i.get_height(),
                (i.get_x() + i.get_width() / 2, i.get_height()),
                ha = 'center', va = 'baseline', fontsize = 12,
                color = 'black', xytext = (0,1),
                textcoords = 'offset pixels')
            locs, labels = plt.xticks()
            plt.setp(labels, rotation=rotation,size=16)
            plt.xlabel('Colúmnas',size=18)
            plt.ylabel('Porcentaje',size=18)
            plot.set_title('Porcentaje de valores perdidos',size=18)

        return missings
    else:
        return False
        """

#get_missings(df)

'\ndef get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):\n    labels,values,percent = list(),list(),list()\n    if df.isna().sum().sum()>0:\n        for column in df.columns:\n            if df[column].isna().sum():\n                labels.append(column)\n                values.append(df[column].isna().sum())\n                percent.append((df[column].isna().sum() / df.shape[0]) * 100)\n        #Make a dataframe \n        missings=pd.DataFrame({\'Colúmnas\':labels,\'Valores faltantes\':values,\'PorcentajePerdido\':percent}).sort_values(by=\'PorcentajePerdido\',ascending=False)\n        \n\n        if plot:\n            plt.figure(figsize=figsize)\n            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title(\'Porcentaje de valores perdidos\',size=22)\n            for i in plot.patches:\n                plot.annotate(i.get_height(),\n                (i.get_x() + i.get_width() / 2, i.get_height()),\n          

Aplicamos la transformacion de los tipos de datos en el dataset de taxis para una mejor categorizacion y eliminamos las columnas irrelevantes.

In [4]:
def t_datime(df):
    df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
    df["hora"]= df.tpep_pickup_datetime.dt.floor("H")

    df.drop(columns=["congestion_surcharge","airport_fee","VendorID","store_and_fwd_flag"], inplace=True)
    return df

def t_int(df):
    df['passenger_count'] = pd.to_numeric(df['passenger_count'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast="integer" )
    df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast="integer" )
    df['payment_type'] = pd.to_numeric(df['payment_type'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    return df

def t_float(df):
    df['trip_distance'] = pd.to_numeric(df['trip_distance'], downcast="float" )
    df['fare_amount'] = pd.to_numeric(df['fare_amount'], downcast="float" )
    df['extra'] = pd.to_numeric(df['extra'], downcast="float" )
    df['mta_tax'] = pd.to_numeric(df['mta_tax'], downcast="float" )
    df['tip_amount'] = pd.to_numeric(df['tip_amount'], downcast="float" )
    df['tolls_amount'] = pd.to_numeric(df['tolls_amount'], downcast="float" )
    df['improvement_surcharge'] = pd.to_numeric(df['improvement_surcharge'], downcast="float" )
    df['total_amount'] = pd.to_numeric(round(df['total_amount'] ))
    return df

def total_transform(df):
    df = t_datime(df)
    df = t_int(df)
    df = t_float(df)
    return df

df =  total_transform(df)


Creamos una mascara del dataframe cada 3 dias

In [5]:
df_Impar = df[df.dayofmonth % 3 == 0]


Sorteamos los value por date y filtramos los que estan fuera del mes

In [7]:
def sorted(df):
    global bol
    bol = 1
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    if bol == 1:
        df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']
        bol = 0
    return df

df_Impar = sorted(df_Impar)

De ahora en mas df_Impar es el df a usar

Vamos a obtener los Boroughs de salida y de llegada.

In [10]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

Trabajamos con el clima

In [12]:
#documentacion mismo nombre del clima en cada update

clima = pd.read_csv("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/new york 2018-01-01 to 2018-02-01.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]
clima.datetime = clima['datetime'].apply(pd.to_datetime)


Transformamos los datos de clima para optimizar rendimiento y chequear que esten bien los formatos

In [14]:
def t_clima():
    clima['temp'] = pd.to_numeric(clima['temp'], downcast="float" )
    clima['feelslike'] = pd.to_numeric(round(clima['feelslike'] ))
    clima['precip'] = pd.to_numeric(clima['precip'], downcast="float" )
    clima['snow'] = pd.to_numeric(clima['snow'], downcast="float" )
    clima['snowdepth'] = pd.to_numeric(clima['snowdepth'], downcast="float" )
    clima["conditions"]=clima["conditions"].astype(str)
    return clima

clima = t_clima()

Aplicamos restriccion por meses dentro del clima

In [16]:
def control_clima(dat,clima):
    #aplicar sort fecha
    clima.sort_values(by='datetime', ascending=True, inplace=True)
    date = dat
    if (clima["datetime"] >= date).any():
        clima = clima[clima["datetime"] >= date]
        date_next= date + relativedelta(months=+1)
        clima = clima[clima["datetime"] <= date_next]
        return clima
    else:
        date+= relativedelta(months=+1)
        return control_clima(dat)

clima = control_clima(dt(2018,1,1),clima)

Creamos columna auxiliar de hora dentro de la tabla clima

In [18]:
#clima in global

def aux ():
    
    clima.rename(columns={"datetime": "hora"},inplace=True)
    return clima

clima = aux()

Se mergearon los datos del clima al df_Impar

In [22]:
df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

IMPUTACION DE VALORES Y ARREGLO DE OUTLIERS

se sacan los outliers para calcular los valores medios del avg_km

In [23]:
#crear df sin outliers.
Q1 = df_Impar["trip_distance"].quantile(0.01)
Q3 = df_Impar["trip_distance"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 
mask = df_Impar["trip_distance"]<outliersSup
df_ImparsinOut = df_Impar[mask]

#se sacaron outliers de trip distance

Q1 = df_ImparsinOut["fare_amount"].quantile(0.01)
Q3 = df_ImparsinOut["fare_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 
mask = df_ImparsinOut["fare_amount"]<outliersSup
df_ImparsinOut = df_ImparsinOut[mask]

#se sacaron outliers de fare amount
Q1 = df_ImparsinOut["tip_amount"].quantile(0.01)
Q3 = df_ImparsinOut["tip_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 
mask = df_ImparsinOut["tip_amount"]<outliersSup
df_ImparsinOut = df_ImparsinOut[mask]

#se sacaron outliers de tip amount
Q1 = df_ImparsinOut["total_amount"].quantile(0.01)
Q3 = df_ImparsinOut["total_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 
mask = df_ImparsinOut["total_amount"]<outliersSup
df_ImparsinOut = df_ImparsinOut[mask]



In [31]:
def millas(df):
  df['km_dollar'] = df['trip_distance']/df['fare_amount']
  return df
  

In [32]:

def arreglo (df_ImparsinOut, ID):
    dfsinceros=df_ImparsinOut[df_ImparsinOut.fare_amount != 0]
    dfsinceros=dfsinceros[dfsinceros.fare_amount>0]
    dfsinceros = dfsinceros[dfsinceros.trip_distance != 0]

    df = dfsinceros[dfsinceros.RatecodeID == ID]
    df = millas(df)
    return df

df1 = arreglo(df_ImparsinOut, 1)
df2 = arreglo(df_ImparsinOut, 2)
df3 = arreglo(df_ImparsinOut, 3)
df4 = arreglo(df_ImparsinOut, 4)
df5 = arreglo(df_ImparsinOut, 5)

avg_km1 =df1['km_dollar'].mean()
avg_km2 =df2['km_dollar'].mean()
avg_km3 =df3['km_dollar'].mean()
avg_km4 =df4['km_dollar'].mean()
avg_km5 =df5['km_dollar'].mean()


Se borran los que tienen cero en trip distance y fare amount

In [34]:

df_Impar["borrar"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]==0  and row["trip_distance"]==0 else 0, axis=1)
df_Impar = df_Impar.drop(df_Impar[df_Impar.borrar == 1].index)
df_Impar = df_Impar.drop(columns="borrar")

In [35]:
def kms(df_Impar, ID, avg_km):
    df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km if row["RatecodeID"]==ID  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
    return df_Impar["trip_distance"]

df_Impar["trip_distance"] = kms(df_Impar, 1, avg_km1)
df_Impar["trip_distance"] = kms(df_Impar, 2, avg_km2)
df_Impar["trip_distance"] = kms(df_Impar, 3, avg_km3)
df_Impar["trip_distance"] = kms(df_Impar, 4, avg_km4)
df_Impar["trip_distance"] = kms(df_Impar, 5, avg_km5)


Se borran las ID innecesarias y los valores negativos, dado que tienen algunas incongruencias en algunas fechas por lo cual no podemos afirmar que son cancelaciones

In [36]:
def neg_values(df_Impar):   
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==6].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==99].index)
    return df_Impar

df_Impar=neg_values(df_Impar)

Buscamos los outliers dentro de las columnas que pueden presentar tales

lo que hay que hacer para imputar valores y tratar outliers es lo siguiente:
sacamos los valores outliers y creamos un df aux. a ese df le borramos los valores con cero de la columna trip distance,
luego calculamos es costo promedio de la milla por cada ratecodeid, lo imputamos en el df completo ( no importa si fareamount es un outlier, porque la imputacion de distancia tambien lo va a ser. por ende es posible que ese viaje termine descartado)
luego se trabaja con las etapas de los outliers, ya sabiendo que en trip distance no hay ceros, pero si outliers.

Etapa 1: marcar los outliers., y las tips no las podemos imputar asique las vamos a imputar con su valor medio en caso de ser outlier.

In [38]:
#Tratamiento de Outliers columna trip_distance
Q1 = df_Impar["trip_distance"].quantile(0.01)
Q3 = df_Impar["trip_distance"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_trip = (Q3 + (1.5*IQR)) 
#se crea la columna marcador
df_Impar["trip_distance_out"] = df_Impar.apply(lambda row: 1 if row["trip_distance"]>outliers_Sup_trip else 0, axis=1)

#se marco trip distance, ahora vamos por fare amount

#Tratamiento de Outliers columna fare_amount
Q1 = df_Impar["fare_amount"].quantile(0.01)
Q3 = df_Impar["fare_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_fare = (Q3 + (1.5*IQR)) 
#se marco fare amount
df_Impar["fare_amount_out"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]>outliers_Sup_fare else 0, axis=1)

#Tratamiento de Outliers columna tip_amount
Q1 = df_Impar["tip_amount"].quantile(0.01)
Q3 = df_Impar["tip_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_tip = (Q3 + (1.5*IQR)) 
#se reemplaza las tips outliers por el valor medio de las tips sin tener encuenta los ceros, la propina media es de 3 dolares.
mask = df_Impar["tip_amount"]<outliers_Sup_tip
ventaCantSinOut = df_Impar[mask]
ventaCantSinOut = ventaCantSinOut[ventaCantSinOut.tip_amount != 0]
cantidadMedia = round(ventaCantSinOut.tip_amount.mean())
df_Impar.loc[(df_Impar['tip_amount'] >= outliers_Sup_tip),"tip_amount"] = cantidadMedia #3

#se marco tip amount, ahora le toca a total amount

#Tratamiento de Outliers columna total_amount
Q1 = df_Impar["total_amount"].quantile(0.01)
Q3 = df_Impar["total_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_total_a = (Q3 + (1.5*IQR)) 
df_Impar["total_amount_out"] = df_Impar.apply(lambda row: 1 if row["tip_amount"]>outliers_Sup_total_a else 0, axis=1)


recordar etapa uno borrar si todos los 3 valores importantes son outliers. las propinas se tratan de otra manera.

etapa 2: si trip distance es outlier, pero fare amount no, podemos imputar el valor como hicimos con los que estaban en cero

In [48]:
def kms(df_Impar, ID, avg_km):
    df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km if row["trip_distance_out"]==1  and row["fare_amount_out"]==0 and row["RatecodeID"]==ID else row["trip_distance"], axis=1)
    return df_Impar["trip_distance"]

df_Impar["trip_distance"] = kms(df_Impar, 1, avg_km1)
df_Impar["trip_distance"] = kms(df_Impar, 2, avg_km2)
df_Impar["trip_distance"] = kms(df_Impar, 3, avg_km3)
df_Impar["trip_distance"] = kms(df_Impar, 4, avg_km4)
df_Impar["trip_distance"] = kms(df_Impar, 5, avg_km4)

#los outliers restantes se borran.
df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance>outliers_Sup_trip].index)

etapa 3, si fare amount es un outlier, pero trip distance no Y NO ES CERO, podriamos imputarle un valor segun su ratecode id

In [53]:
def kms(df_Impar, ID, avg_km):
    df_Impar["fare_amount"] = df_Impar.apply(lambda row: row["trip_distance"]/avg_km if row["fare_amount_out"]==1  and row["trip_distance"]!=0 and row["RatecodeID"]==ID else row["trip_distance"], axis=1)
    return df_Impar["trip_distance"]

df_Impar["trip_distance"] = kms(df_Impar, 1, avg_km1)
df_Impar["trip_distance"] = kms(df_Impar, 2, avg_km2)
df_Impar["trip_distance"] = kms(df_Impar, 3, avg_km3)
df_Impar["trip_distance"] = kms(df_Impar, 4, avg_km4)
df_Impar["trip_distance"] = kms(df_Impar, 5, avg_km4)

#se dropean los que quedaron ouliers
df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount>outliers_Sup_fare].index)

etapa 4: si fare amount es cero, pero trip distance no, tenemos que imputar los valores de igual manera que si fuese un outlier. 

como hay solo 6 datos en cero, los borramos por ser insignificantes.

In [56]:
df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount==0].index)

etapa 5: si total_amount es un outlier, deberiamos imputar el valor segun su trip distance segun su ratecode id

In [60]:
def kms(df_Impar, ID, avg_km):
    df_Impar["total_amount"] = df_Impar.apply(lambda row: ((row["trip_distance"]/avg_km)+row["extra"]+row["mta_tax"]+row["tip_amount"]+row["tolls_amount"]+row["improvement_surcharge"]) if row["total_amount_out"]==1  and row["trip_distance"]!=0 and row["RatecodeID"]==ID else row["trip_distance"], axis=1)
    return df_Impar["trip_distance"]

df_Impar["trip_distance"] = kms(df_Impar, 1, avg_km1)
df_Impar["trip_distance"] = kms(df_Impar, 2, avg_km2)
df_Impar["trip_distance"] = kms(df_Impar, 3, avg_km3)
df_Impar["trip_distance"] = kms(df_Impar, 4, avg_km4)
df_Impar["trip_distance"] = kms(df_Impar, 5, avg_km4)

In [62]:
df_Impar.to_parquet('C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet',engine="pyarrow")

6 verificar outliers de duracion